# Convert pickle files to ASCII text files for CAMEL

Pickles:

In [1]:
ls /data/SatDragModelValidation/data/outputs_clean/icesat2/Paper2023_icesat2_assessment/results_fixedCD

20181109000200_20181123000000__24hrArcs__FixedCD2.5__msis2.csv
20181109000200_20181123000000__24hrArcs__FixedCD2.5__tiegcm_oc.csv
ctipe_oc_2week_CD2.5.pkl
dtm2020_o_2week_CD2.5.pkl
gitm_2week_CD2.5.pkl
hasdm_oc_2week_CD2.5.pkl
jb2008_2week_CD2.5.pkl
msis2_2week_CD2.5.pkl
tiegcm_oc_2week_CD2.5.pkl


- Need to match the time cadence of the density values to that of the in-track residuals.
- save data as ascii-text files
- save data as JSON format

In [2]:
from pygeodyn.PYGEODYN import Pygeodyn
from pygeodyn.pygeodyn_plot import *
import datetime


run_list = [
#                 'msis2',
#                 'tiegcm_oc',
#                 'ctipe_oc',
                'jb2008',
#                 'dtm2020_o',
# #                 'gitm',
#                 'hasdm_oc',
           ]

plot_dir='/data/SatDragModelValidation/notebooks/Paper2023_icesat2_assessment/plots/'

dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    if i =='msis2':
        run_dict[i]={}
        run_dict[i]['num'] = 5
        run_dict[i]['model_path'] = None

    if i =='dtm2020_o':
        run_dict[i]={}
        run_dict[i]['num'] = 3
        run_dict[i]['model_path'] = None

    if i =='jb2008':
        run_dict[i]={}
        run_dict[i]['num'] = 1
        run_dict[i]['model_path'] = None

    ### PHYSICAL MODELS
    if i =='tiegcm_oc':
        run_dict[i]={}
        run_dict[i]['num'] = 0
        run_dict[i]['model_path'] = dir_modeldat+'/tiegcm/icesat2_oct2018_jan2019'
    if i =='ctipe_oc':
        run_dict[i]={}
        run_dict[i]['num']=4
        run_dict[i]['model_path'] = dir_modeldat+'/ctipe/icesat2_oct2018_jan2019'
    if i =='gitm':
        run_dict[i]={}
        run_dict[i]['num']=4
        run_dict[i]['model_path'] = dir_modeldat+'/gitm/icesat2_oct2018_jan2019'
    if i =='hasdm_oc':
        run_dict[i]={}
        run_dict[i]['num'] = 2
        run_dict[i]['model_path'] = dir_modeldat+'/hasdm/vishal_icesat2_oct2018_jan2019'
print(run_dict)

{'jb2008': {'num': 1, 'model_path': None}}


## Fixed CD-2.5 Case

In [3]:
%load_ext autoreload
%autoreload 2
# from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

import pickle 
import os
import pandas as pd
import time
from datetime import datetime,timedelta



for run_dir in ['results_2week_ScalingFactor',
#                 'results_fixedCD',
                'results_withScalingFactor']:

    global_path    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'icesat2/Paper2023_icesat2_assessment/'+run_dir+'/'

    if run_dir == 'results_fixedCD':
        pickleName = "_2week_CD2.5.pkl"
        fileID     = 'FixedCD2.5'
        
    elif run_dir == 'results_withScalingFactor':
        pickleName = "_2week_DragScaleFactor.pkl"
        fileID     = '24hrDragScaleFactor'

    
    elif run_dir == 'results_2week_ScalingFactor':
        pickleName = "_2weeks_TwoWeekDragScaleFactor.pkl"
        fileID     = 'TwoWeekDragScaleFactor'

    else:
        import sys
        sys.exit()

    obj = {}
    for i,model in enumerate(run_list):     
        ### Load the data if the pickles exist
        print()
        print()
        gc_collect()

        pickle_file = global_path+model+pickleName

        filehandler = open(pickle_file, 'rb') 
        obj[model] = pickle.load(filehandler)
        filehandler.close()
        print('Loaded data from pickle... ', model)




    ### Declare some setup parameters:
    len_date        = 19
    len_lon_lat_alt =  9
    len_rho         = 15
    len_resids      = 12
    len_f107        = 5
    len_kp          = 5

    for imodel in run_list:
        start = time.time()
        print('\n','Saving Text file for ', imodel)

        obj_m1 = obj[imodel]
        CAMELfilename = global_path+'20181109000200_20181123000000__24hrArcs__'+fileID+'__'+imodel+'.csv'    
        print('   filename: ',CAMELfilename)

        if imodel == 'msis2':
            modelname = "NRLMSIS 2.0"
        elif imodel == 'dtm2020_o':
            modelname = "DTM2020 (oper.)"
        elif imodel == 'jb2008':
            modelname = "JB2008"
        elif imodel == 'tiegcm_oc':
            modelname = "TIEGCMv2.0"
        elif imodel == 'hasdm_oc':
            modelname = "SET HASDM"
        elif imodel == 'ctipe_oc':
            modelname = "CTIPev4.1"


        ### Delete the existing files so they append cleanly
        if os.path.exists(CAMELfilename):
            os.remove(CAMELfilename)




        f = open(CAMELfilename, "w")
        f.write("\n")
        f.close()

        ### Write to file
        with open(CAMELfilename, 'r+') as file:


            ### Manually write the header units
            header_units =\
                    f"{'(UTC)'.rjust(len_date-1,' ') }"\
                    +f"  {'(deg)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(deg)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(m)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(kg/m^3)'.rjust(len_rho,' ')}"\
                    +f"  {'(m)'.rjust(len_resids,' ')}"\
                    +f"  {'(m)'.rjust(len_resids,' ')}"\
        #             +f"  {'(sfu)'.rjust(len_f107,' ')}"\
        #             +f"  {'(n/a)'.rjust(len_kp,' ')}"\

            #### Manually write the header field names
            header_names =\
                    f"{'Date'.rjust(len_date-1,' ') }"\
                    +f"  {'Lon'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Lat'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Alt'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Rho'.rjust(len_rho,' ')}"\
                    +f"  {'T_PSO'.rjust(len_resids,' ')}"\
                    +f"  {'T_OrbFit'.rjust(len_resids,' ')}"\
        #             +f"  {'f107'.rjust(15,' ')}"\
        #             +f"  {'kp'.rjust(15,' ')}"\

            #### Manually write the detailed header description
            header_meta = \
                    f'''### Model Assessment---ICESat-2 Orbit Fit using {modelname}
### -----------------------
###     Satellite: ICESat2
###     Epoch:     +start____ {'2018-11-09 00:02:00'} 
###                +stop_____ {'2018-11-23 00:00:00'}
###     Last modified: {datetime.now()-timedelta(hours=7)}
###
### Sources
### -------
###     ICESat-2 Orbit Fit using {modelname}
###     ICESat-2 Precise Science Orbit (PSO)
###
###
#{header_units}
#{header_names}
### %eoh
'''

            file.write(header_meta)  


            ### Loop through the arcs and append the datasets for each arc into CSV files
            for ii,arc in enumerate(obj_m1['global_params']['arc_input'][:]):

                arc =arc+'.01'
                print(arc)

                ### ----------------------------------------------------------------------------------
                ### SAVE DENSITY at same cadence as the T RESIDUALS
                ##
                ####   Find the datetime values in the density dataset that 
                ####       are closest to those in the residual datset.
                ####   The two sets have different cadences 
                ####       density is the integration step size (10sec)
                ####       orbfil is the output size (2min)

                def nearest(items, pivot):
                    return min(items, key=lambda x: abs(x - pivot))

                timesList = pd.to_datetime(obj_m1['Density'][arc]['Date'])
                vals  = np.arange(obj_m1['Density'][arc].index[0],obj_m1['Density'][arc].index[-1]+1)
                df = obj_m1['Density'][arc].set_index('Date',drop=False ) 
                df['i_vals'] = vals
                for itime,timeCheck in enumerate(pd.to_datetime(obj_m1['OrbitResids'][arc]['data_PCE']['Date'])):

                    date_nearstart = nearest(pd.to_datetime(df['i_vals'].index), timeCheck)
                    index_date = df['i_vals'][date_nearstart].min()

                    time_near = timesList[index_date]


                    if np.abs((timeCheck - time_near).total_seconds()) < 10:

                        date_list = timeCheck
                        den_list = obj_m1['Density'][arc]['rho (kg/m**3)'][index_date]
                        intrack_PCE = obj_m1['OrbitResids'][arc]['data_PCE']['T'][itime]
                        intrack_Orbfile = obj_m1['OrbitResids'][arc]['data_orbfil']['T'][itime]
                        lon_list = obj_m1['Density'][arc]['Lon'][index_date]
                        lat_list = obj_m1['Density'][arc]['Lat'][index_date]
                        alt_list = obj_m1['Density'][arc]['Height (meters)'][index_date]

                    else:
                        print('THE TWO TIMES ARE FAR APART')
                        print('     timeCheck, time_near: ', timeCheck, time_near)
                        den_list = np.nan
                        date_list = index_date
                        intrack_PCE = obj_m1['OrbitResids'][arc]['data_PCE']['T'][itime]
                        intrack_Orbfile = obj_m1['OrbitResids'][arc]['data_orbfil']['T'][itime]
                        lon_list = np.nan
                        lat_list = np.nan
                        alt_list = np.nan


                    #### Manually write each row of the data.
                    row =   f"{pd.to_datetime(date_list).strftime(format='%Y-%m-%d %H:%M:%S')}"\
                        +f"  {lon_list:9.3f}"\
                        +f"  {lat_list:9.3f}"\
                        +f"  {alt_list:9.3f}"\
                        +f"  {den_list:15.6e}"\
                        +f"  {intrack_PCE:12.5f}"\
                        +f"  {intrack_Orbfile:12.5f}"\
                        +f"\n"
                    file.write(row)
    #
        end = time.time()
        elapsed = end - start
        #
        print()
        # print(f'       indxes: 0 -',indx,'') 
        print(f'       Processed file in : ',np.round(elapsed,5),'secs', f"({np.round(elapsed,5)/60} minutes)") 




Loaded data from pickle...  jb2008

 Saving Text file for  jb2008
   filename:  /data/SatDragModelValidation/data/outputs_clean/icesat2/Paper2023_icesat2_assessment/results_2week_ScalingFactor/20181109000200_20181123000000__24hrArcs__TwoWeekDragScaleFactor__jb2008.csv
2018.313.01
2018.314.01
2018.315.01
2018.316.01


KeyboardInterrupt: 